In [1]:
import pandas as pd
import os
from prophet import Prophet
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

C:\Users\82107\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 적당운동 5%

### 데이터 불러오기

In [2]:
os.chdir('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\data\\scenario')

In [3]:
# 적당운동 5%
data1 = pd.read_csv('젊음_적당_5_normal.csv') 
data2 = pd.read_csv('중년_적당_5_normal.csv') 
data3 = pd.read_csv('장년_적당_5_normal.csv') 

In [4]:
# datetime 형태로 변경
data1['update_time'] = pd.to_datetime(data1['update_time'])
data2['update_time'] = pd.to_datetime(data2['update_time'])
data3['update_time'] = pd.to_datetime(data3['update_time'])

In [5]:
print(data1.shape)
print(data2.shape)
print(data3.shape)

(1350, 2001)
(1350, 2001)
(1350, 2001)


In [6]:
data1.head()

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,104,0,164,1,114,0,115,0,152,...,126,0,126,0,90,0,93,0,128,0
1,2023-10-01 00:10:00,99,0,100,0,159,0,145,0,151,...,142,0,125,0,136,0,138,0,135,0
2,2023-10-01 00:20:00,145,0,140,0,161,0,132,0,122,...,135,0,138,0,140,0,117,0,135,0
3,2023-10-01 00:30:00,129,0,109,0,134,0,122,0,116,...,131,0,113,0,125,0,91,0,141,0
4,2023-10-01 00:40:00,132,0,111,0,96,0,126,0,166,...,103,0,130,0,123,0,132,0,108,0


### 적당운동 5%_젊음

In [7]:
# 최종 결과 데이터 만들기
pred1 = pd.DataFrame()

In [8]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df1 = data1[['update_time', f'heart_rate{i}', f'event{i}']]
    # columns 명 변경
    df1.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df1[:1008]
    test = df1[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred1[f'heart_rate{i}'] = performance[['y']]
    pred1[f'event{i}'] = test[[f'event{i}']]
    pred1[f'pred_y{i}'] = performance[f'pred_y{i}']
    

20:46:46 - cmdstanpy - INFO - Chain [1] start processing
20:46:47 - cmdstanpy - INFO - Chain [1] done processing
20:46:47 - cmdstanpy - INFO - Chain [1] start processing
20:46:47 - cmdstanpy - INFO - Chain [1] done processing
20:46:48 - cmdstanpy - INFO - Chain [1] start processing
20:46:48 - cmdstanpy - INFO - Chain [1] done processing
20:46:49 - cmdstanpy - INFO - Chain [1] start processing
20:46:49 - cmdstanpy - INFO - Chain [1] done processing
20:46:50 - cmdstanpy - INFO - Chain [1] start processing
20:46:50 - cmdstanpy - INFO - Chain [1] done processing
20:46:51 - cmdstanpy - INFO - Chain [1] start processing
20:46:51 - cmdstanpy - INFO - Chain [1] done processing
20:46:51 - cmdstanpy - INFO - Chain [1] start processing
20:46:52 - cmdstanpy - INFO - Chain [1] done processing
20:46:52 - cmdstanpy - INFO - Chain [1] start processing
20:46:52 - cmdstanpy - INFO - Chain [1] done processing
20:46:53 - cmdstanpy - INFO - Chain [1] start processing
20:46:53 - cmdstanpy - INFO - Chain [1]

In [9]:
pred1

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,106,0,0,133,0,0,113,0,0,87,...,0,122,0,0,112,0,0,107,0,0
1,128,0,0,125,0,0,103,0,0,111,...,0,117,0,0,143,0,0,138,0,0
2,141,0,0,84,0,0,102,0,0,76,...,0,86,0,0,92,0,0,108,0,0
3,122,0,0,93,0,0,104,0,0,97,...,0,113,0,0,131,0,0,113,0,0
4,154,0,1,102,0,0,107,0,0,132,...,0,134,0,0,135,0,0,111,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,126,0,0,151,0,0,116,0,0,86,...,1,128,0,0,134,0,0,137,0,0
338,108,0,0,118,0,0,136,0,0,114,...,0,142,0,0,111,0,0,102,0,0
339,126,0,0,138,0,0,161,0,1,145,...,0,92,0,0,139,0,0,129,0,0
340,181,1,1,124,0,0,125,0,0,105,...,0,103,0,0,101,0,0,92,0,0


In [10]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [11]:
predict1 = pd.concat([test1, pred1], axis = 1)

In [12]:
predict1

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,106,0,0,133,0,0,113,0,0,...,0,122,0,0,112,0,0,107,0,0
1,2023-10-08 00:10:00,128,0,0,125,0,0,103,0,0,...,0,117,0,0,143,0,0,138,0,0
2,2023-10-08 00:20:00,141,0,0,84,0,0,102,0,0,...,0,86,0,0,92,0,0,108,0,0
3,2023-10-08 00:30:00,122,0,0,93,0,0,104,0,0,...,0,113,0,0,131,0,0,113,0,0
4,2023-10-08 00:40:00,154,0,1,102,0,0,107,0,0,...,0,134,0,0,135,0,0,111,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,126,0,0,151,0,0,116,0,0,...,1,128,0,0,134,0,0,137,0,0
338,2023-10-10 08:20:00,108,0,0,118,0,0,136,0,0,...,0,142,0,0,111,0,0,102,0,0
339,2023-10-10 08:30:00,126,0,0,138,0,0,161,0,1,...,0,92,0,0,139,0,0,129,0,0
340,2023-10-10 08:40:00,181,1,1,124,0,0,125,0,0,...,0,103,0,0,101,0,0,92,0,0


In [13]:
predict1.to_csv('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\predict_result\\normal5_청년_적장_Prophet_predict.csv')

### 적당운동 5%_중년

In [14]:
# 최종 결과 데이터 만들기
pred2 = pd.DataFrame()

In [15]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df2 = data2[['update_time', f'heart_rate{i}', f'event{i}']]
    # columns 명 변경
    df2.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df2[:1008]
    test = df2[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred2[f'heart_rate{i}'] = performance[['y']]
    pred2[f'event{i}'] = test[[f'event{i}']]
    pred2[f'pred_y{i}'] = performance[f'pred_y{i}']
    

21:01:06 - cmdstanpy - INFO - Chain [1] start processing
21:01:06 - cmdstanpy - INFO - Chain [1] done processing
21:01:06 - cmdstanpy - INFO - Chain [1] start processing
21:01:07 - cmdstanpy - INFO - Chain [1] done processing
21:01:07 - cmdstanpy - INFO - Chain [1] start processing
21:01:07 - cmdstanpy - INFO - Chain [1] done processing
21:01:08 - cmdstanpy - INFO - Chain [1] start processing
21:01:08 - cmdstanpy - INFO - Chain [1] done processing
21:01:09 - cmdstanpy - INFO - Chain [1] start processing
21:01:09 - cmdstanpy - INFO - Chain [1] done processing
21:01:09 - cmdstanpy - INFO - Chain [1] start processing
21:01:10 - cmdstanpy - INFO - Chain [1] done processing
21:01:10 - cmdstanpy - INFO - Chain [1] start processing
21:01:10 - cmdstanpy - INFO - Chain [1] done processing
21:01:11 - cmdstanpy - INFO - Chain [1] start processing
21:01:11 - cmdstanpy - INFO - Chain [1] done processing
21:01:12 - cmdstanpy - INFO - Chain [1] start processing
21:01:12 - cmdstanpy - INFO - Chain [1]

In [16]:
pred2.head()

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,95,0,0,120,0,0,101,0,0,77,...,0,110,0,0,100,0,0,96,0,0
1,115,0,0,113,0,0,91,0,0,99,...,0,105,0,0,129,0,0,125,0,0
2,128,0,0,74,0,0,91,0,0,66,...,0,76,0,0,82,0,0,97,0,0
3,110,0,0,82,0,0,93,0,0,86,...,0,101,0,0,119,0,0,101,0,0
4,140,0,1,91,0,0,96,0,0,119,...,0,121,0,0,122,0,0,99,0,0


In [17]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [18]:
predict2 = pd.concat([test1, pred2], axis = 1)

In [28]:
predict2.head()

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,95,0,0,120,0,0,101,0,0,...,0,110,0,0,100,0,0,96,0,0
1,2023-10-08 00:10:00,115,0,0,113,0,0,91,0,0,...,0,105,0,0,129,0,0,125,0,0
2,2023-10-08 00:20:00,128,0,0,74,0,0,91,0,0,...,0,76,0,0,82,0,0,97,0,0
3,2023-10-08 00:30:00,110,0,0,82,0,0,93,0,0,...,0,101,0,0,119,0,0,101,0,0
4,2023-10-08 00:40:00,140,0,1,91,0,0,96,0,0,...,0,121,0,0,122,0,0,99,0,0


In [29]:
predict2.to_csv('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\predict_result\\normal5_중년_적장_Prophet_predict.csv')

### 적당운동 5%_노년

In [21]:
# 최종 결과 데이터 만들기
pred3 = pd.DataFrame()

In [22]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df3 = data3[['update_time', f'heart_rate{i}', f'event{i}']]
    # columns 명 변경
    df3.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df3[:1008]
    test = df3[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred3[f'heart_rate{i}'] = performance[['y']]
    pred3[f'event{i}'] = test[[f'event{i}']]
    pred3[f'pred_y{i}'] = performance[f'pred_y{i}']
    

21:16:19 - cmdstanpy - INFO - Chain [1] start processing
21:16:19 - cmdstanpy - INFO - Chain [1] done processing
21:16:19 - cmdstanpy - INFO - Chain [1] start processing
21:16:20 - cmdstanpy - INFO - Chain [1] done processing
21:16:20 - cmdstanpy - INFO - Chain [1] start processing
21:16:20 - cmdstanpy - INFO - Chain [1] done processing
21:16:21 - cmdstanpy - INFO - Chain [1] start processing
21:16:21 - cmdstanpy - INFO - Chain [1] done processing
21:16:22 - cmdstanpy - INFO - Chain [1] start processing
21:16:22 - cmdstanpy - INFO - Chain [1] done processing
21:16:23 - cmdstanpy - INFO - Chain [1] start processing
21:16:23 - cmdstanpy - INFO - Chain [1] done processing
21:16:23 - cmdstanpy - INFO - Chain [1] start processing
21:16:24 - cmdstanpy - INFO - Chain [1] done processing
21:16:24 - cmdstanpy - INFO - Chain [1] start processing
21:16:24 - cmdstanpy - INFO - Chain [1] done processing
21:16:25 - cmdstanpy - INFO - Chain [1] start processing
21:16:25 - cmdstanpy - INFO - Chain [1]

In [23]:
pred3

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,84,0,0,109,0,0,90,0,0,66,...,0,98,0,0,89,0,0,85,0,0
1,104,0,0,101,0,0,80,0,0,88,...,0,94,0,0,117,0,0,113,0,0
2,116,0,0,63,0,0,80,0,0,55,...,0,65,0,0,71,0,0,86,0,0
3,99,0,0,71,0,0,82,0,0,75,...,0,90,0,0,107,0,0,90,0,0
4,128,0,1,80,0,0,84,0,0,107,...,0,109,0,0,110,0,0,88,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,102,0,0,125,0,0,93,0,0,65,...,1,103,0,0,109,0,0,112,0,0
338,86,0,0,95,0,0,111,0,0,91,...,0,117,0,0,88,0,0,79,0,0
339,102,0,0,113,0,0,135,1,1,120,...,0,71,0,0,114,0,0,105,0,0
340,153,1,1,100,0,0,101,0,0,82,...,0,81,0,0,79,0,0,70,0,0


In [24]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [25]:
predict3 = pd.concat([test1, pred3], axis = 1)

In [26]:
predict3.head()

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,84,0,0,109,0,0,90,0,0,...,0,98,0,0,89,0,0,85,0,0
1,2023-10-08 00:10:00,104,0,0,101,0,0,80,0,0,...,0,94,0,0,117,0,0,113,0,0
2,2023-10-08 00:20:00,116,0,0,63,0,0,80,0,0,...,0,65,0,0,71,0,0,86,0,0
3,2023-10-08 00:30:00,99,0,0,71,0,0,82,0,0,...,0,90,0,0,107,0,0,90,0,0
4,2023-10-08 00:40:00,128,0,1,80,0,0,84,0,0,...,0,109,0,0,110,0,0,88,0,0


In [27]:
predict3.to_csv('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\predict_result\\normal5_노년_적당_Prophet_predict.csv')

# 적당운동 10%

### 데이터 불러오기

In [30]:
# 적당운동 10%
data4 = pd.read_csv('젊음_적당_10_normal.csv') 
data5 = pd.read_csv('중년_적당_10_normal.csv') 
data6 = pd.read_csv('장년_적당_10_normal.csv') 

In [31]:
# datetime 형태로 변경
data4['update_time'] = pd.to_datetime(data4['update_time'])
data5['update_time'] = pd.to_datetime(data5['update_time'])
data6['update_time'] = pd.to_datetime(data6['update_time'])

In [32]:
print(data4.shape)
print(data5.shape)
print(data6.shape)

(1350, 2001)
(1350, 2001)
(1350, 2001)


In [33]:
data4.head()

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,100,0,175,1,113,0,114,0,160,...,128,0,127,0,82,0,87,0,130,0
1,2023-10-01 00:10:00,93,0,95,0,169,1,151,0,159,...,147,0,126,0,140,0,143,0,139,0
2,2023-10-01 00:20:00,151,0,145,0,171,1,135,0,123,...,138,0,142,0,145,0,116,0,139,0
3,2023-10-01 00:30:00,132,0,106,0,137,0,123,0,115,...,134,0,111,0,127,0,83,0,147,0
4,2023-10-01 00:40:00,135,0,109,0,90,0,128,0,178,...,99,0,132,0,123,0,135,0,105,0


### 적당운동 10%_청년

In [34]:
# 최종 결과 데이터 만들기
pred4 = pd.DataFrame()

In [35]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df4 = data4[['update_time', f'heart_rate{i}', f'event{i}']]
    # columns 명 변경
    df4.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df4[:1008]
    test = df4[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred4[f'heart_rate{i}'] = performance[['y']]
    pred4[f'event{i}'] = test[[f'event{i}']]
    pred4[f'pred_y{i}'] = performance[f'pred_y{i}']
    

21:31:15 - cmdstanpy - INFO - Chain [1] start processing
21:31:15 - cmdstanpy - INFO - Chain [1] done processing
21:31:15 - cmdstanpy - INFO - Chain [1] start processing
21:31:15 - cmdstanpy - INFO - Chain [1] done processing
21:31:16 - cmdstanpy - INFO - Chain [1] start processing
21:31:16 - cmdstanpy - INFO - Chain [1] done processing
21:31:16 - cmdstanpy - INFO - Chain [1] start processing
21:31:16 - cmdstanpy - INFO - Chain [1] done processing
21:31:16 - cmdstanpy - INFO - Chain [1] start processing
21:31:16 - cmdstanpy - INFO - Chain [1] done processing
21:31:17 - cmdstanpy - INFO - Chain [1] start processing
21:31:17 - cmdstanpy - INFO - Chain [1] done processing
21:31:17 - cmdstanpy - INFO - Chain [1] start processing
21:31:17 - cmdstanpy - INFO - Chain [1] done processing
21:31:18 - cmdstanpy - INFO - Chain [1] start processing
21:31:18 - cmdstanpy - INFO - Chain [1] done processing
21:31:18 - cmdstanpy - INFO - Chain [1] start processing
21:31:18 - cmdstanpy - INFO - Chain [1]

In [36]:
pred4

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,103,0,0,136,0,0,111,0,0,79,...,0,122,0,0,109,0,0,104,0,0
1,130,0,0,127,0,0,98,0,0,108,...,0,117,0,0,148,0,0,142,0,0
2,146,0,0,75,1,0,97,0,0,64,...,0,78,1,0,85,0,0,106,0,0
3,123,0,0,86,0,0,101,0,0,91,...,0,111,0,0,134,0,0,111,0,0
4,162,1,1,97,0,0,104,0,0,135,...,0,137,0,0,139,0,0,108,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,128,0,0,158,0,0,115,0,0,78,...,1,129,0,0,137,0,0,141,0,0
338,105,0,0,118,0,0,139,0,0,112,...,0,148,0,0,108,0,0,97,0,0
339,128,0,0,143,0,0,171,1,1,152,...,0,85,0,0,144,0,0,131,0,0
340,196,1,1,125,0,0,126,0,0,101,...,0,99,0,0,96,0,0,84,0,0


In [37]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [38]:
predict4 = pd.concat([test1, pred4], axis = 1)

In [39]:
predict4.head()

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,103,0,0,136,0,0,111,0,0,...,0,122,0,0,109,0,0,104,0,0
1,2023-10-08 00:10:00,130,0,0,127,0,0,98,0,0,...,0,117,0,0,148,0,0,142,0,0
2,2023-10-08 00:20:00,146,0,0,75,1,0,97,0,0,...,0,78,1,0,85,0,0,106,0,0
3,2023-10-08 00:30:00,123,0,0,86,0,0,101,0,0,...,0,111,0,0,134,0,0,111,0,0
4,2023-10-08 00:40:00,162,1,1,97,0,0,104,0,0,...,0,137,0,0,139,0,0,108,0,0


In [40]:
predict4.to_csv('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\predict_result\\normal10_청년_적당_Prophet_predict.csv')

### 적당운동 10%_중년

In [41]:
# 최종 결과 데이터 만들기
pred5 = pd.DataFrame()

In [42]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df5 = data5[['update_time', f'heart_rate{i}', f'event{i}']]
    # columns 명 변경
    df5.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df5[:1008]
    test = df5[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred5[f'heart_rate{i}'] = performance[['y']]
    pred5[f'event{i}'] = test[[f'event{i}']]
    pred5[f'pred_y{i}'] = performance[f'pred_y{i}']
    

21:39:45 - cmdstanpy - INFO - Chain [1] start processing
21:39:45 - cmdstanpy - INFO - Chain [1] done processing
21:39:45 - cmdstanpy - INFO - Chain [1] start processing
21:39:45 - cmdstanpy - INFO - Chain [1] done processing
21:39:46 - cmdstanpy - INFO - Chain [1] start processing
21:39:46 - cmdstanpy - INFO - Chain [1] done processing
21:39:46 - cmdstanpy - INFO - Chain [1] start processing
21:39:46 - cmdstanpy - INFO - Chain [1] done processing
21:39:46 - cmdstanpy - INFO - Chain [1] start processing
21:39:47 - cmdstanpy - INFO - Chain [1] done processing
21:39:47 - cmdstanpy - INFO - Chain [1] start processing
21:39:47 - cmdstanpy - INFO - Chain [1] done processing
21:39:47 - cmdstanpy - INFO - Chain [1] start processing
21:39:47 - cmdstanpy - INFO - Chain [1] done processing
21:39:48 - cmdstanpy - INFO - Chain [1] start processing
21:39:48 - cmdstanpy - INFO - Chain [1] done processing
21:39:48 - cmdstanpy - INFO - Chain [1] start processing
21:39:48 - cmdstanpy - INFO - Chain [1]

In [44]:
pred5

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,92,0,0,123,0,0,100,0,0,71,...,0,110,0,0,98,0,0,93,0,0
1,117,0,0,114,0,0,88,0,0,97,...,0,105,0,0,134,0,0,129,0,0
2,132,0,0,67,1,0,87,0,0,57,...,0,69,1,0,76,0,0,95,0,0
3,111,0,0,76,0,0,90,0,0,81,...,0,100,0,0,121,0,0,100,0,0
4,147,0,1,87,0,0,93,0,0,122,...,0,124,0,0,125,0,0,97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,115,0,0,143,0,0,103,0,0,69,...,1,117,0,0,124,0,0,127,0,0
338,94,0,0,106,0,0,126,0,0,101,...,0,134,0,0,97,0,0,87,0,0
339,115,0,0,129,0,0,155,1,1,137,...,0,76,0,0,130,0,0,118,0,0
340,178,1,1,112,0,0,113,0,0,90,...,0,89,0,0,86,0,0,75,0,0


In [45]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [46]:
predict5 = pd.concat([test1, pred5], axis = 1)

In [47]:
predict5.head()

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,92,0,0,123,0,0,100,0,0,...,0,110,0,0,98,0,0,93,0,0
1,2023-10-08 00:10:00,117,0,0,114,0,0,88,0,0,...,0,105,0,0,134,0,0,129,0,0
2,2023-10-08 00:20:00,132,0,0,67,1,0,87,0,0,...,0,69,1,0,76,0,0,95,0,0
3,2023-10-08 00:30:00,111,0,0,76,0,0,90,0,0,...,0,100,0,0,121,0,0,100,0,0
4,2023-10-08 00:40:00,147,0,1,87,0,0,93,0,0,...,0,124,0,0,125,0,0,97,0,0


In [48]:
predict5.to_csv('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\predict_result\\normal10_중년_적당_Prophet_predict.csv')

### 적당운동 10%_노년

In [49]:
# 최종 결과 데이터 만들기
pred6 = pd.DataFrame()

In [50]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df6 = data6[['update_time', f'heart_rate{i}', f'event{i}']]
    # columns 명 변경
    df6.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df6[:1008]
    test = df6[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred6[f'heart_rate{i}'] = performance[['y']]
    pred6[f'event{i}'] = test[[f'event{i}']]
    pred6[f'pred_y{i}'] = performance[f'pred_y{i}']
    

22:06:45 - cmdstanpy - INFO - Chain [1] start processing
22:06:45 - cmdstanpy - INFO - Chain [1] done processing
22:06:45 - cmdstanpy - INFO - Chain [1] start processing
22:06:46 - cmdstanpy - INFO - Chain [1] done processing
22:06:46 - cmdstanpy - INFO - Chain [1] start processing
22:06:46 - cmdstanpy - INFO - Chain [1] done processing
22:06:46 - cmdstanpy - INFO - Chain [1] start processing
22:06:46 - cmdstanpy - INFO - Chain [1] done processing
22:06:47 - cmdstanpy - INFO - Chain [1] start processing
22:06:47 - cmdstanpy - INFO - Chain [1] done processing
22:06:47 - cmdstanpy - INFO - Chain [1] start processing
22:06:47 - cmdstanpy - INFO - Chain [1] done processing
22:06:47 - cmdstanpy - INFO - Chain [1] start processing
22:06:47 - cmdstanpy - INFO - Chain [1] done processing
22:06:48 - cmdstanpy - INFO - Chain [1] start processing
22:06:48 - cmdstanpy - INFO - Chain [1] done processing
22:06:48 - cmdstanpy - INFO - Chain [1] start processing
22:06:48 - cmdstanpy - INFO - Chain [1]

In [51]:
pred6

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,81,0,0,111,0,0,89,0,0,60,...,0,99,0,0,87,0,0,82,0,0
1,105,0,0,103,0,0,77,0,0,86,...,0,93,0,0,122,0,0,117,0,0
2,120,0,0,56,1,0,76,0,0,46,...,0,59,1,0,65,0,0,84,0,0
3,99,0,0,66,0,0,79,0,0,70,...,0,89,0,0,109,0,0,89,0,0
4,135,1,1,76,0,0,82,0,0,110,...,0,112,0,0,113,0,0,86,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,104,0,0,131,0,0,92,0,0,59,...,1,105,0,0,112,0,0,115,0,0
338,83,0,0,95,0,0,114,0,0,89,...,0,122,0,0,86,0,0,76,0,0
339,104,0,0,117,0,0,143,1,1,125,...,0,65,0,0,118,0,0,107,0,0
340,165,1,1,101,0,0,102,0,0,79,...,0,78,0,0,75,0,0,64,0,0


In [52]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [53]:
predict6 = pd.concat([test1, pred6], axis = 1)

In [54]:
predict6.head()

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,81,0,0,111,0,0,89,0,0,...,0,99,0,0,87,0,0,82,0,0
1,2023-10-08 00:10:00,105,0,0,103,0,0,77,0,0,...,0,93,0,0,122,0,0,117,0,0
2,2023-10-08 00:20:00,120,0,0,56,1,0,76,0,0,...,0,59,1,0,65,0,0,84,0,0
3,2023-10-08 00:30:00,99,0,0,66,0,0,79,0,0,...,0,89,0,0,109,0,0,89,0,0
4,2023-10-08 00:40:00,135,1,1,76,0,0,82,0,0,...,0,112,0,0,113,0,0,86,0,0


In [55]:
predict6.to_csv('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation\\predict_result\\normal10_노년_적당_Prophet_predict.csv')

# result

In [56]:
import math

In [57]:
def performance(TP, TN, FP, FN):
    
    N = TP+TN+FP+FN
    
    Sensitivity=(TP/(TP+FN))*100

    Specificity=(1-FP/(TN+FP))*100

    Accuracy=((TP+TN)/(TP+TN+FP+FN))*100

    PPV= (TP/(TP+FP))*100 

    NPV= (TN/(TN+FN))*100 
    
    result = {
        'Sensitivity': [Sensitivity],
        'Specificity': [Specificity],
        'Accuracy':[Accuracy],
        'PPV':[PPV],
        'NPV':[NPV]
    }
    
    result_2 = pd.DataFrame(result)
    
    return result_2

### 적당운동 5%

- 청년

In [91]:
prophet_result = pd.DataFrame()

for i in range(1,1001):

    TP = len(pred1[(pred1[f'event{i}']==1)&(pred1[f'pred_y{i}']==1)])
    TN = len(pred1[(pred1[f'event{i}']==0)&(pred1[f'pred_y{i}']==0)])
    FN = len(pred1[(pred1[f'event{i}']==1)&(pred1[f'pred_y{i}']==0)])
    FP = len(pred1[(pred1[f'event{i}']==0)&(pred1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)
    
    prophet_result = pd.concat([prophet_result, result_i])
    
prophet_result.index = [f"result{i}" for i in range(1, 1001)]

prophet_result.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
result1,100.0,98.181818,98.245614,66.666667,100.000000
result2,87.5,98.159509,97.660819,70.000000,99.378882


In [92]:
mean_prophet = pd.DataFrame(index=['mean'])

mean_prophet['mean_sens'] = prophet_result['Sensitivity'].mean()
mean_prophet['mean_spec'] = prophet_result['Specificity'].mean()
mean_prophet['mean_accu'] = prophet_result['Accuracy'].mean()
mean_prophet['mean_ppv'] = prophet_result['PPV'].mean()
mean_prophet['mean_npv'] = prophet_result['NPV'].mean()

mean_prophet

,mean_sens,mean_spec,mean_accu,mean_ppv,mean_npv
mean,95.364953,98.190795,98.077485,69.450294,99.802949


- 중년

In [93]:
prophet_result2 = pd.DataFrame()

for i in range(1,1001):

    TP = len(pred2[(pred2[f'event{i}']==1)&(pred2[f'pred_y{i}']==1)])
    TN = len(pred2[(pred2[f'event{i}']==0)&(pred2[f'pred_y{i}']==0)])
    FN = len(pred2[(pred2[f'event{i}']==1)&(pred2[f'pred_y{i}']==0)])
    FP = len(pred2[(pred2[f'event{i}']==0)&(pred2[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)
    
    prophet_result2 = pd.concat([prophet_result2, result_i])
    
prophet_result2.index = [f"result{i}" for i in range(1, 1001)]

prophet_result2.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
result1,100.0,97.878788,97.953216,63.157895,100.000000
result2,87.5,98.466258,97.953216,73.684211,99.380805


In [94]:
mean_prophet2 = pd.DataFrame(index=['mean'])

mean_prophet2['mean_sens'] = prophet_result2['Sensitivity'].mean()
mean_prophet2['mean_spec'] = prophet_result2['Specificity'].mean()
mean_prophet2['mean_accu'] = prophet_result2['Accuracy'].mean()
mean_prophet2['mean_ppv'] = prophet_result2['PPV'].mean()
mean_prophet2['mean_npv'] = prophet_result2['NPV'].mean()

mean_prophet2

,mean_sens,mean_spec,mean_accu,mean_ppv,mean_npv
mean,95.525546,98.19078,98.084503,69.309013,99.811071


- 노년

In [66]:
prophet_result3 = pd.DataFrame()

for i in range(1,1001):

    TP = len(pred3[(pred3[f'event{i}']==1)&(pred3[f'pred_y{i}']==1)])
    TN = len(pred3[(pred3[f'event{i}']==0)&(pred3[f'pred_y{i}']==0)])
    FN = len(pred3[(pred3[f'event{i}']==1)&(pred3[f'pred_y{i}']==0)])
    FP = len(pred3[(pred3[f'event{i}']==0)&(pred3[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)
    
    prophet_result3 = pd.concat([prophet_result3, result_i])
    
prophet_result3.index = [f"result{i}" for i in range(1, 1001)]

prophet_result3.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
result1,100.000000,98.170732,98.245614,70.000000,100.000000
result2,82.352941,98.461538,97.660819,73.684211,99.071207


In [67]:
mean_prophet3 = pd.DataFrame(index=['mean'])

mean_prophet3['mean_sens'] = prophet_result3['Sensitivity'].mean()
mean_prophet3['mean_spec'] = prophet_result3['Specificity'].mean()
mean_prophet3['mean_accu'] = prophet_result3['Accuracy'].mean()
mean_prophet3['mean_ppv'] = prophet_result3['PPV'].mean()
mean_prophet3['mean_npv'] = prophet_result3['NPV'].mean()

mean_prophet3

,mean_sens,mean_spec,mean_accu,mean_ppv,mean_npv
mean,94.184172,98.401946,98.223684,73.123253,99.739081


### 적당운동 10%

- 청년

In [68]:
prophet_result4 = pd.DataFrame()

for i in range(1,1001):

    TP = len(pred4[(pred4[f'event{i}']==1)&(pred4[f'pred_y{i}']==1)])
    TN = len(pred4[(pred4[f'event{i}']==0)&(pred4[f'pred_y{i}']==0)])
    FN = len(pred4[(pred4[f'event{i}']==1)&(pred4[f'pred_y{i}']==0)])
    FP = len(pred4[(pred4[f'event{i}']==0)&(pred4[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)
    
    prophet_result4 = pd.concat([prophet_result4, result_i])
    
prophet_result4.index = [f"result{i}" for i in range(1, 1001)]

prophet_result4.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
result1,52.777778,100.000000,95.02924,100.000000,94.736842
result2,52.941176,99.675325,95.02924,94.736842,95.046440


In [69]:
mean_prophet4 = pd.DataFrame(index=['mean'])

mean_prophet4['mean_sens'] = prophet_result4['Sensitivity'].mean()
mean_prophet4['mean_spec'] = prophet_result4['Specificity'].mean()
mean_prophet4['mean_accu'] = prophet_result4['Accuracy'].mean()
mean_prophet4['mean_ppv'] = prophet_result4['PPV'].mean()
mean_prophet4['mean_npv'] = prophet_result4['NPV'].mean()

mean_prophet4

,mean_sens,mean_spec,mean_accu,mean_ppv,mean_npv
mean,54.495757,99.950157,95.366959,99.299638,95.146372


- 중년

In [70]:
prophet_result5 = pd.DataFrame()

for i in range(1,1001):

    TP = len(pred5[(pred5[f'event{i}']==1)&(pred5[f'pred_y{i}']==1)])
    TN = len(pred5[(pred5[f'event{i}']==0)&(pred5[f'pred_y{i}']==0)])
    FN = len(pred5[(pred5[f'event{i}']==1)&(pred5[f'pred_y{i}']==0)])
    FP = len(pred5[(pred5[f'event{i}']==0)&(pred5[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)
    
    prophet_result5 = pd.concat([prophet_result5, result_i])
    
prophet_result5.index = [f"result{i}" for i in range(1, 1001)]

prophet_result5.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
result1,59.375000,99.677419,95.906433,95.0,95.962733
result2,57.575758,100.000000,95.906433,100.0,95.665635


In [71]:
mean_prophet5 = pd.DataFrame(index=['mean'])

mean_prophet5['mean_sens'] = prophet_result5['Sensitivity'].mean()
mean_prophet5['mean_spec'] = prophet_result5['Specificity'].mean()
mean_prophet5['mean_accu'] = prophet_result5['Accuracy'].mean()
mean_prophet5['mean_ppv'] = prophet_result5['PPV'].mean()
mean_prophet5['mean_npv'] = prophet_result5['NPV'].mean()

mean_prophet5

,mean_sens,mean_spec,mean_accu,mean_ppv,mean_npv
mean,60.634765,99.892766,96.367836,98.409742,96.261205


- 노년

In [89]:
prophet_result6 = pd.DataFrame()

for i in range(1,1001):

    TP = len(pred6[(pred6[f'event{i}']==1)&(pred6[f'pred_y{i}']==1)])
    TN = len(pred6[(pred6[f'event{i}']==0)&(pred6[f'pred_y{i}']==0)])
    FN = len(pred6[(pred6[f'event{i}']==1)&(pred6[f'pred_y{i}']==0)])
    FP = len(pred6[(pred6[f'event{i}']==0)&(pred6[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN)
    
    prophet_result6 = pd.concat([prophet_result6, result_i])
    
prophet_result6.index = [f"result{i}" for i in range(1, 1001)]

prophet_result6.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV
result1,51.428571,100.0,95.029240,100.0,94.753086
result2,57.575758,100.0,95.906433,100.0,95.665635


In [90]:
mean_prophet6 = pd.DataFrame(index=['mean'])

mean_prophet6['mean_sens'] = prophet_result6['Sensitivity'].mean()
mean_prophet6['mean_spec'] = prophet_result6['Specificity'].mean()
mean_prophet6['mean_accu'] = prophet_result6['Accuracy'].mean()
mean_prophet6['mean_ppv'] = prophet_result6['PPV'].mean()
mean_prophet6['mean_npv'] = prophet_result6['NPV'].mean()

mean_prophet6

,mean_sens,mean_spec,mean_accu,mean_ppv,mean_npv
mean,57.340065,99.928483,95.861111,98.982357,95.690048
